# Natural Gas Price Analysis & Forecast


This notebook:
- Loads monthly natural gas prices  
- Visualizes historical trends  
- Fits a time-series model  
- Estimates price for any given date  
- Extrapolates prices one year into the future


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing


In [ ]:

# Load dataset
df = pd.read_csv('/mnt/data/Nat_Gas.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')
df.set_index('Date', inplace=True)
df.head()


In [ ]:

# Plot historical prices
plt.figure(figsize=(12,6))
plt.plot(df.index, df['Price'])
plt.title('Monthly Natural Gas Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid()
plt.show()


In [ ]:

# Seasonal decomposition
decomp = seasonal_decompose(df['Price'], model='additive', period=12)
decomp.plot()
plt.show()


In [ ]:

# Train Holt-Winters model
model = ExponentialSmoothing(
    df['Price'], trend='add', seasonal='add', seasonal_periods=12
)
fit = model.fit()


In [ ]:

# Forecast next 12 months
forecast = fit.forecast(12)
future_dates = pd.date_range(df.index[-1] + pd.offsets.MonthEnd(1), periods=12, freq='M')

plt.figure(figsize=(12,6))
plt.plot(df.index, df['Price'], label='Historical')
plt.plot(future_dates, forecast, label='Forecast', linestyle='--')
plt.legend()
plt.title('Natural Gas Price Forecast')
plt.grid()
plt.show()


In [ ]:

# Price estimation function
def estimate_price(input_date):
    input_date = pd.to_datetime(input_date)
    if input_date in df.index:
        return float(df.loc[input_date, 'Price'])
    all_series = pd.concat([df['Price'], forecast])
    all_series.index = pd.concat([df.index, future_dates])
    return float(np.interp(
        input_date.toordinal(),
        all_series.index.map(pd.Timestamp.toordinal),
        all_series.values
    ))

# Example
estimate_price('2025-03-31')
